#Flow✈
1. Go through the flow once before executing any code snippet
2. Upload Train.csv file from the xipped folder to google colab
3. Run the sections one after the other in the order- Data Import-- Training Data Preprocessing -- Training model -- Test case single -- User interface.
###Link for user interface - https://penchant-analytics.anvil.app/

#Data Import

In [1]:
import pandas as pd
import numpy as np
import csv 
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
train = pd.read_csv('Train.csv', delimiter =',', nrows=12000)
train = train[train['text'].notnull()]
df=train
#df.head(1)

#Training Data Preprocessing

In [3]:
import re
def clean_text(text):

    text = text.lower()
    text = re.sub(r'@[a-zA-Z0-9_]+', '', text)   
    text = re.sub(r'https?://[A-Za-z0-9./]+', '', text)   
    text = re.sub(r'www.[^ ]+', '', text)  
    text = re.sub(r'[a-zA-Z0-9]*www[a-zA-Z0-9]*com[a-zA-Z0-9]*', '', text)  
    text = re.sub(r'[^a-zA-Z]', ' ', text)   
    text = [token for token in text.split() if len(token) > 2]
    text = ' '.join(text)
    return text

df['text'] = df['text'].apply(clean_text)


In [4]:
#The most important operations
from textblob import TextBlob
df['polarity'] = df.apply(lambda x: TextBlob(x['text']).sentiment.polarity, axis=1)
df['subjectivity'] = df.apply(lambda x: TextBlob(x['text']).sentiment.subjectivity, axis=1)

In [5]:
import nltk.corpus
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
df['text'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.tokenize import TreebankWordTokenizer

cvec = TfidfVectorizer(max_features=10000, norm = 'l1', 
                       lowercase=True, smooth_idf=False, 
                       sublinear_tf=False, ngram_range=(1,4), 
                       tokenizer=TreebankWordTokenizer().tokenize)

df_info = pd.DataFrame(cvec.fit_transform(df['text']).todense())
df_info.columns = ['Product_Description_Top_' + str(c) for c in df_info.columns]
df = pd.concat([df, df_info], axis=1)

In [7]:
df['numerics'] = df['text'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))
df['upper'] = df['text'].apply(lambda x: len([x for x in x.split() if x.isupper()]))

In [8]:
# import nltk
# nltk.download('stopwords')
# nltk.download('vader_lexicon')

In [9]:
import string
punctuation=string.punctuation
df['word_count']=df['text'].apply(lambda x: len(x.split(" ")))
df['char_count'] = df['text'].str.len()
def avg_word(sentence):
    words = sentence.split()
    return (sum(len(word) for word in words)/(len(words)+1))

df['avg_word'] = df['text'].apply(lambda x: avg_word(x))   #average length of a word


# from nltk.corpus import stopwords
# stop = stopwords.words('english')                   

# df['stopwords'] = df['text'].apply(lambda x: len([x for x in x.split() if x in stop]))       #stop words in sentence
df['word_density'] = df['char_count'] / (df['word_count']+1)                                                #Total no of characters/word count

In [10]:
j=[]
for i in df['text']:
  j.append(len(i))
df['len']=j

In [11]:
del df['text']
train_df = df[df['label'].isnull()==False]

In [12]:
X = train_df.drop(labels=['label'], axis=1)
y = train_df['label'].values
X

,polarity,subjectivity,Product_Description_Top_0,Product_Description_Top_1,Product_Description_Top_2,Product_Description_Top_3,Product_Description_Top_4,Product_Description_Top_5,Product_Description_Top_6,Product_Description_Top_7,Product_Description_Top_8,Product_Description_Top_9,Product_Description_Top_10,Product_Description_Top_11,Product_Description_Top_12,Product_Description_Top_13,Product_Description_Top_14,Product_Description_Top_15,Product_Description_Top_16,Product_Description_Top_17,Product_Description_Top_18,Product_Description_Top_19,Product_Description_Top_20,Product_Description_Top_21,Product_Description_Top_22,Product_Description_Top_23,Product_Description_Top_24,Product_Description_Top_25,Product_Description_Top_26,Product_Description_Top_27,Product_Description_Top_28,Product_Description_Top_29,Product_Description_Top_30,Product_Description_Top_31,Product_Description_Top_32,Product_Description_Top_33,Product_Description_Top_34,Product_Description_Top_35,Product_Description_Top_36,Product_Description_Top_37,...,Product_Description_Top_9967,Product_Description_Top_9968,Product_Description_Top_9969,Product_Description_Top_9970,Product_Description_Top_9971,Product_Description_Top_9972,Product_Description_Top_9973,Product_Description_Top_9974,Product_Description_Top_9975,Product_Description_Top_9976,Product_Description_Top_9977,Product_Description_Top_9978,Product_Description_Top_9979,Product_Description_Top_9980,Product_Description_Top_9981,Product_Description_Top_9982,Product_Description_Top_9983,Product_Description_Top_9984,Product_Description_Top_9985,Product_Description_Top_9986,Product_Description_Top_9987,Product_Description_Top_9988,Product_Description_Top_9989,Product_Description_Top_9990,Product_Description_Top_9991,Product_Description_Top_9992,Product_Description_Top_9993,Product_Description_Top_9994,Product_Description_Top_9995,Product_Description_Top_9996,Product_Description_Top_9997,Product_Description_Top_9998,Product_Description_Top_9999,numerics,upper,word_count,char_count,avg_word,word_density,len
0,0.123287,0.635741,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.020239,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,84,606,6.152941,7.129412,606
1,0.120678,0.478799,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,158,1084,5.830189,6.817610,1084
2,-0.052083,0.374915,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0293,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.007645,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,94,609,5.431579,6.410526,609
3,-0.135714,0.616667,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.044407,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,33,212,5.294118,6.235294,212
4,0.123256,0.591011,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.013095,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,97,635,5.500000,6.479592,635
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [13]:
y

array([0, 0, 0, ..., 0, 1, 1])

#Training model

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_cv, y_train, y_cv = train_test_split(X, y, test_size=0.10, random_state=101, stratify=y)

In [15]:
X_train.shape, y_train.shape, X_cv.shape, y_cv.shape

((10800, 10009), (10800,), (1200, 10009), (1200,))

In [16]:
from sklearn.metrics import log_loss

In [17]:
#Multiclass classification using Lightgbm
import lightgbm as lgb
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_cv, label=y_cv)

param = {'objective': 'multiclass',
         'num_class': 2,
         'boosting': 'gbdt',  
         'metric': 'multi_logloss',
         'learning_rate': 0.01, 
         'num_iterations': 2400,
         'num_leaves': 31,
         'max_depth': -1,
         'min_data_in_leaf': 15,
         'bagging_fraction':0.9,
         'bagging_freq': 2,
         'feature_fraction': 0.9,
         'lambda_l2': 0.9,
         'min_data_per_group': 75,
         'max_bin': 255,
         'is_unbalance':True
         }

clf = lgb.train(params=param, 
                early_stopping_rounds=200,
                verbose_eval=100,
                train_set=train_data,
                valid_sets=[test_data])

y_pred = clf.predict(X_cv)

/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 200 rounds.
[100]	valid_0's multi_logloss: 0.487728
[200]	valid_0's multi_logloss: 0.428434
[300]	valid_0's multi_logloss: 0.398011
[400]	valid_0's multi_logloss: 0.375157
[500]	valid_0's multi_logloss: 0.359206
[600]	valid_0's multi_logloss: 0.346601
[700]	valid_0's multi_logloss: 0.337731
[800]	valid_0's multi_logloss: 0.330383
[900]	valid_0's multi_logloss: 0.323922
[1000]	valid_0's multi_logloss: 0.318723
[1100]	valid_0's multi_logloss: 0.31479
[1200]	valid_0's multi_logloss: 0.312088
[1300]	valid_0's multi_logloss: 0.309416
[1400]	valid_0's multi_logloss: 0.30721
[1500]	valid_0's multi_logloss: 0.305537
[1600]	valid_0's multi_logloss: 0.303596
[1700]	valid_0's multi_logloss: 0.302503
[1800]	valid_0's multi_logloss: 0.301311
[1900]	valid_0's multi_logloss: 0.300287
[2000]	valid_0's multi_logloss: 0.299393
[2100]	valid_0's multi_logloss: 0.298668
[2200]	valid_0's multi_logloss: 0.298255
[2300]	valid_0's multi_logloss: 0.29805
[2400]

In [18]:
log_loss(y_cv, y_pred)

0.29770452367010863

In [19]:
del df

#Test case -single
##User input is required in this section 
##Sample inputs from test file

##Negative

when i first read about "berlin am meer" i didn't expect much. but i thought with the right people, the right locations, the right music and fashion you could at least make a trivial movie about the hip berlin everyone seems to be talking about. but eissler failed, it's so ridiculously unauthentic. it's a complete misrepresentation of what it is going on in berlin's so called scene. of course it's not all about hippness, but you should expect more from a movie that's being sold as "the definite berlin movie".<br /><br />and apart from all the credibility stuff, it really is a bad movie. mediocre acting and a rather boring plot. interestingly some of the actors have proved in other movies that they are actually quite talented. so it really must be poor directing skills.<br /><br />don't bother watching "berlin am meer" unless you are 17, come from some small town in western Germany and want to move to the big city after you finished school. then you might actually find it enjoyable and totally cool.

##Positive
My interest was raised as I was flipping through and saw the name Iphigenia. My name is Eugenia so I thought OK, lets see what this is. I am so glad I stayed on the channel. What a wonderful, wonderful story. Drama, sadness, some over the top acting but a wonderful time to be had. I watch this and it makes me sad for all the drivel the movie industry puts out and these beautiful little gems get passed over. Give Iphigenia a try and I hope you will enjoy it as much as I did. I have even gotten my children (27, 25, 20 and 17) to enjoy it. It starts slow, however, the drama builds and you will be drawn in to the story. Watching this lovely film made me want to shroud myself in more Greek tragedy and pathos.


In [20]:
def data_prep(text):
    data = {'text':[unseen]}
    test = pd.DataFrame(data)

    test['text'] = test['text'].apply(clean_text)
    test['polarity'] = test.apply(lambda x: TextBlob(x['text']).sentiment.polarity, axis=1)
    test['subjectivity'] = test.apply(lambda x: TextBlob(x['text']).sentiment.subjectivity, axis=1)
    test['text'] = test['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

    df2=train.append(test,ignore_index=True)
    df2_info = pd.DataFrame(cvec.fit_transform(df2['text']).todense())
    df2_info.columns = ['Product_Description_Top_' + str(c) for c in df2_info.columns]
    df2 = pd.concat([df2, df2_info], axis=1)
    #df2.tail()

    test = df2[df2['label'].isnull()==True]
    del df2
    test['numerics'] = test['text'].apply(lambda x: len([x for x in x.split() if x.isdigit()]));
    test['upper'] = test['text'].apply(lambda x: len([x for x in x.split() if x.isupper()]));

    test['word_count']=test['text'].apply(lambda x: len(str(x).split(" ")))
    test['char_count'] = test['text'].str.len()
    test['avg_word'] = test['text'].apply(lambda x: avg_word(x))   #average length of a word
    #test['stopwords'] = test['text'].apply(lambda x: len([x for x in x.split() if x in stop]))       #stop words in sentence
    test['word_density'] = test['char_count'] / (test['word_count']+1)                                                #Total no of characters/word count

    j=[]
    for i in test['text']:
        j.append(len(i))
    test['len']=j

    del test['text']
    return test

In [21]:
unseen = input("Enter the review to be classified: ")
test = data_prep(unseen)

y_pred = clf.predict(test)
if y_pred[0][0]>y_pred[0][1]:
    print('The sentiment is negative')
else:
    print('The sentiment is positive')
del y_pred
del test

Enter the review to be classified: My interest was raised as I was flipping through and saw the name Iphigenia. My name is Eugenia so I thought OK, lets see what this is. I am so glad I stayed on the channel. What a wonderful, wonderful story. Drama, sadness, some over the top acting but a wonderful time to be had. I watch this and it makes me sad for all the drivel the movie industry puts out and these beautiful little gems get passed over. Give Iphigenia a try and I hope you will enjoy it as much as I did. I have even gotten my children (27, 25, 20 and 17) to enjoy it. It starts slow, however, the drama builds and you will be drawn in to the story. Watching this lovely film made me want to shroud myself in more Greek tragedy and pathos.
The sentiment is positive


#User Interface

In [ ]:
!pip install anvil-uplink
import anvil.server
anvil.server.connect("EUYKULBCEZ3KD467Z6DU7XEW-I545NQUFEWIXQZ3M")


@anvil.server.callable
def predict(text2classify):
    test = data_prep(unseen)
    y_pred = clf.predict(test)
    if y_pred[0][0]>y_pred[0][1]:
        result = 'Negative'
    else:
        result = 'Positive'
    return result

anvil.server.wait_forever()